In [2]:
import cv2 as cv
import numpy as np
import os

In [3]:
# Ruta de la carpeta con las imágenes
directorio = "./images"

# Lista para guardar las imágenes
imagenes = []

# Recorremos los archivos del directorio
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # Verificamos que sea un archivo y termine en una extensión de imagen
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa)
        if img is not None:
            imagenes.append(img)
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes)} imágenes en blanco y negro")

Se leyeron 7 imágenes en blanco y negro


In [5]:
# --- Cargar template ---
template = cv.imread("./template/pattern.png")

# --- Preprocesar template ---
template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
template_blur = cv.GaussianBlur(template_gray, (3,3), 0)
template_edges = cv.Canny(template_blur, 40, 160)

for img in imagenes:
    h, w = template_edges.shape
    best_val = -1
    best_loc = None
    best_scale = None
    best_result = None

    # --- Recorrer pirámide de escalas ---
    for scale in np.linspace(0.25, 3, 40):
        resized = cv.resize(img, None, fx=scale, fy=scale)
        if resized.shape[0] < h or resized.shape[1] < w:
            continue  # saltar si el template no entra en la imagen

        # Preprocesar imagen escalada
        img_gray = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
        img_blur = cv.GaussianBlur(img_gray, (3,3), 0)
        img_edges = cv.Canny(img_blur, 40, 160)

        # Matching
        result = cv.matchTemplate(img_edges, template_edges, cv.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        if max_val > best_val:
            best_val = max_val
            best_loc = max_loc
            best_scale = scale
            best_result = resized.copy()

    # --- Verificar antes de dibujar ---
    if best_loc is not None:
        top_left = best_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv.rectangle(best_result, top_left, bottom_right, (255, 0, 0), 2)
        print(f"Mejor coincidencia: {best_val:.4f} en escala {best_scale:.2f}")
        cv.imshow("Mejor resultado", best_result)
        cv.waitKey(0)
        cv.destroyAllWindows()
    else:
        print("No se encontró coincidencia.")

Mejor coincidencia: 0.0919 en escala 0.32
Mejor coincidencia: 0.3035 en escala 2.29
Mejor coincidencia: 0.0732 en escala 2.93
Mejor coincidencia: 0.0834 en escala 2.86
Mejor coincidencia: 0.1096 en escala 0.74
Mejor coincidencia: 0.1584 en escala 2.51
Mejor coincidencia: 0.1045 en escala 1.45
